In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("sample_submission.csv")
df.head()

,key_id,word
0,9000003627287624,The_Eiffel_Tower airplane donut
1,9000010688666847,The_Eiffel_Tower airplane donut
2,9000023642890129,The_Eiffel_Tower airplane donut
3,9000038588854897,The_Eiffel_Tower airplane donut
4,9000052667981386,The_Eiffel_Tower airplane donut


In [ ]:
from vis import *

draw_random_images(train_X, train_y, word_encoder)
